#Housing Case Study - End to End ML Process

**Business Need** - Make real-estate investment decision for a large corporation.

**Sub-Problem** - Predict the median house price in a block (smallest census group in US) given other demographic and geographic information.

**For use in** - Other downstream models that need median house price in a block. For example, they may consider a greenfield project. So they need to know what a new block would be.

**Data Science Problem** - Predict a value (the median house price in a block) given several features (other demographic and geographic information.)

It is a supervised learning problem.

**Dataset** - California Census Data with Median Housing Price

Case Study Prepared by modifying Chapter 2 of Geron's textbook by Prof. Deepak Subramani, CDS

Instructions:
* Please go through the code, most of the blocks are self explanatory.
* You will gain more practice with each part through the course.
* Don't worry even if you don;t understand everything on day 1.
* Your goal should be to execute a project like this at the end of Module 2.

In [ ]:
# Generic Imports, Plot Beautification

import numpy as np
import os
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rc('axes',labelsize=14)
mpl.rc('xtick',labelsize=12)
mpl.rc('ytick',labelsize=12)

In [ ]:
#Setup Figure Saving

fig_dir_path = os.path.join(".","figs")
os.makedirs(fig_dir_path, exist_ok=True)

def save_this(fig_id, tight_layout=True, fig_extension = "png", resolution = 300):
  fig_full_path = os.path.join(fig_dir_path,fig_id+'.'+fig_extension)
  print("saving figure",fig_id)
  if tight_layout:
    plt.tight_layout()
  plt.savefig(fig_full_path,dpi=resolution,format=fig_extension)


#Data#

  - Download data from the following link into your local system: https://www.dropbox.com/s/hwnd5h0f0in0jt0/housing.csv?dl=0

  - Upload the file from your local system to this notebook by clicking the 'Upload to session storage' icon in the left panel

  - Wait for the file to be 100% uploaded.

  - You will be able to see the name of the file in the left panel once it is uploaded into Colab

  - The following code will import it (Make sure the filename matches the name of the uploaded file).


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
%%bash

ls             #lists the directory files


**Familiarize yourself with the tabular data**

Load data from CSV to a dataframe and look at the first 5 rows.

Look at what types are these columns (find their info)



In [ ]:
import io                                              #io is a set of top level reader functions accessed like pandas .read_csv() that generally returns a pandas object
df = pd.read_csv(io.BytesIO(uploaded['housing.csv']))  #BytesIO are methods that manipulate bytes data in memory which is used for binary data
df.head()  #To see first 5 records

Q: Find the number of rows and columns of the data frame.

Task: Read the column names and try to understand what they mean. If you don't understand, ask the upstream team who provided the data. Ask the data engineer who created the pipeline.

For this case study, we will provide you the meaning of each column:

longitude, latitute: Geographical location of each block ("district")

population: Total Population in that block.

households: Total Households in that block.

housing_mean_age: Median age of the households in that block.

total_room: Total number of rooms in all the houses in that block.

total_bedroomns: Total number of bedrooms in all the houses in that block.

median_income: Median income of the households in that block.

median_house_value: Median value of houses in that block (this is our Target)

ocean_proximity: Categorical variable describing the proximity of that block to the ocean.

In [ ]:
df.shape     # To check number of rows and columns in the data

In [ ]:
df.info()

We see that Columns 0-8 are float64 and 9 is an object.

#Exploratory Data Analysis#

Q: Find the number of unique items in the column "ocean_proximity"

In [ ]:
df["ocean_proximity"].value_counts()    #value_counts returns the count of the unique values of the perticular column

In [ ]:
df["ocean_proximity"].unique()   # returns the unique values of series object

Q: Find the descriptive statistics of the numerical values in the table

In [ ]:
df.describe() #Finds the mean, std, min, quartiles, max of all numerical values

We have explored both numerical and categorical variables and got an understanding of the scale of the data

There are some issues to notice:



1.   Is median_income strange?
2.   Why do we have total_bedrooms and total_rooms? Won't bedrooms per house or rooms per house be more useful?

3.   How to deal with Categorical Data?

4. Are longitude and latitude useful?

Indeed, median_income is strange. First it seems to be not in simple dollars. Surely, income is not 3.87 dollars on average. And there seems to be a max of 15.0001. You have to enquire with the data provider on why this is the case.

On asking, the upstream team tells you that it is actually in 10k USD and there is an upper cut-off of 150k USD, meaning median_income is never above 15k USD in this table.

You immediately notice that median_house_value is also bounded at 500001. So you enquire and understand that median_house_value is indeed in USD and an upper bound is set.

This latter finding is problematic for you. median_house_value is our Target, and the training data has artifically imposed a limit on our target. It is a warning sign to be careful.

Now let us look at the distribution of all the numerical values.

Task: Beautify the histogram plot. Show it in a readable format. Play around with the bin sizes and get it into an interpretable format.

In [ ]:
df.hist(bins=25,figsize=(20,15))   # hist funtion gives the distribution of the numeric variables

In [ ]:
save_this("df_value_histogram")

**Interpret**

Histogram of longitude and latitude show that most blocks are concentrated in a few locations. We need geographical plots to see where these locations are.

Median Housing Age seems to have a upper limit as well at 52. Is it problematic?

Median House Value has a chunk at the upper limit. This can be a problem. One option is to remove it completely from the dataset, and clarify to downstream teams that our system cannot predict median_house_value beyond 500000 USD.

Here, we retain and proceed.

#Train-Test Split

Before we proceed further we must split the dataset into a train (really the train-validate) and test set. The test set is then usually removed from the working environment and saved separately.

We looked at the data attributes and histogram before Train-Test Split is to ensure that our dataset is representative of all targets.

From our above analysis, we do not suspect any non-representativeness (except for >500k USD house price).

Let us do a simple test-train split using sklearn.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df,test_size=0.2,random_state=42)

train_test_split can work on X, y as well. If the input is X,y instead of df, then train_test_split returns X_train, X_test, y_train and y_test.

The above splitting is usually enough. But in special cases, we employ what is called as "Stratified Shuffle Split".

Let us say that you did train_test_split as above, and your upstream data colleague invited you for 11 am coffee. At the coffee shop he tells you, "Pay attention to median_income. I think that median_income is extremely important for predicting median_housing_value."

So you come back and decide to explore median_income more.

In [ ]:
df["median_income"].hist(bins=50)

In [ ]:
df["median_income"].describe()

You see that bulk of the data is between 1.5 (15k) and 6 (60k). This also confirms your understanding of median incomes. So you decide to create bins and categorize income to marginal (0-1.5), low (1.5-3), middle (3-4.5), upper middle (4.5-6), upper classes (6+).

Task: Use pandas function pd.cut and create the above categorization.

Q: What is the count of each category?



In [ ]:
df["income_cat"] = pd.cut(df["median_income"],bins=[0., 1.5, 3., 4.5, 6., np.inf ], labels=[1,2,3,4,5])

In [ ]:
df["income_cat"].value_counts()

Now we have 5 strata in the dataset. Ideally, we want the train_test_split to sample in the specified ratio from each strata. For this, we use sklearn's stratify keyword arg of train_test_split.

Note: Earlier we said that dealing with categorical variables is hard. But now we converted a continuos value to a categorical variable. This conversion helps us in the present application to perform stratifies sampling for train_test_split.

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42) #No stratification
df_train_strat, df_test_strat = train_test_split(df, test_size=0.2, random_state=42, stratify=df["income_cat"])

Let us see the difference in value_counts proportions in df_train and df_train_strat

In [ ]:
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

compare_props_train = pd.DataFrame({
    "Overall": income_cat_proportions(df),
    "Stratified": income_cat_proportions(df_train_strat),
    "Random": income_cat_proportions(df_train),
}).sort_index()
compare_props_train["Rand. %error"] = 100 * compare_props_train["Random"] / compare_props_train["Overall"] - 100
compare_props_train["Strat. %error"] = 100 * compare_props_train["Stratified"] / compare_props_train["Overall"] - 100

In [ ]:
compare_props_train

Now, ideally we must remove df_test_strat from the notebook and save it as a test.csv.

This is an precaution to avoid any kind of accidental data leakage.

If you follow the non-notebook mechanism (as in AAMPL), a separate cross-validation split is used and train and test csv are built as a first step.

Here, we trust that we will be careful in avoiding data leakage (i.e., test set is somehow used during train-validate).

# Explore train data for ML modeling

In [ ]:
df_train_strat.drop("income_cat", axis=1, inplace=True) # income_cat was created for stratified split. Its use is over. Let us remove it.
df_test_strat.drop("income_cat", axis=1, inplace=True)
housing = df_train_strat.copy() # Create a copy to avoid data leakage, and accidental deletion.

**Geo Plots**

Since this is a geographical dataset, we can plot the target on a map to visualize. Such visualization can sometimes trigger ideas on what features to use.

We do not plot it here. But you can explore geographical plotting with several resources


*   https://jakevdp.github.io/PythonDataScienceHandbook/04.13-geographic-data-with-basemap.html
*   https://geopandas.org/en/stable/docs/user_guide/mapping.html
*   https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73



**Correlation**

Let us see the correlation of all the features with the target

In [ ]:
corr_mat = housing.corr()
corr_mat["median_house_value"].sort_values(ascending=False)

Plot the heat map of the correlation matrix

In [ ]:
sns.heatmap(corr_mat)

In [ ]:
corr_mat

We see that median_income has the highest correlation with median_house_value as our friend had suggested. We already did a smart move - of stratified sampling.

We also note that latitude has a negative correlation with house value. Its implication is that moving north reduces the house value.

Population has a slightly negative correlation as expected.

Let us look at a few more visualizations of the data

Task 1: Plot the scatter of median_house_vale, median_income, total_rooms, housing_median_age using seaborn

In [ ]:
sns.scatterplot(data=housing, x="median_income", y="total_rooms", hue="median_house_value")

In [ ]:
cols_needed = ["median_income","total_rooms","housing_median_age","median_house_value"]
sns.pairplot(data=housing[cols_needed], hue="median_house_value") #This will take some time

This phase is to fire your brain to think of possible combinations for feature engieering. You must use your business knowledge (or process, science, domain knowledge) to figure out what features are important for this problem.

**Feature Engineering**

The strange issue here is that total_rooms and total_bedrooms are not as correlated with median_house_value. In fact, our intutition suggests that 3BHK is more pricier than 2BHK etc. So we should look at rooms and bedroom per household.

Let us do that feature engineering.

Task: Add new features of rooms_per_house and bedrooms_per_house

Q: What is the correlation of the new features to the target.

In [ ]:
housing["rooms_per_house"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_house"] = housing["total_bedrooms"]/housing["households"]
corr_mat2 = housing.corr()
corr_mat2["median_house_value"].sort_values(ascending=False)

Not much changed. Let us try different combinations of rooms.


*   bedrooms_per_room
*   population_per_house



In [ ]:
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_house"] = housing["population"]/housing["households"]
corr_mat3 = housing.corr()
corr_mat3["median_house_value"].sort_values(ascending=False)

OK! Bedrooms_per_room is more correlated (-vely) with house value. Apparently houses with more bedrooms cost less. Somehow people seem to value houses with more non-bedrooms here.

Message: We have identified that median_income, bedrooms_per_room and rooms_per_house (or total_rooms), housing_median_age seem to be good predictors for house_value. Latitude is also correlated, but it seems to be strange to include that as a predictor.

We still have to worry about missing data and handling categorical data.

Now we are entering the stage of preparing the data for ML.

Remember we need X and y for ML.

# Prepare Data for ML

In [ ]:
hml = df_train_strat.drop("median_house_value", axis=1) # drop labels for training set
hml_labels = df_train_strat["median_house_value"].copy()
# Note: Here we did not use the housing df as above. We have modified housing df for exploration purposes. We will use that learning in creating an automatic data processing pipeline soon.

**Handle Missing Data**

Q: How many data points are missing? What featues are missing?

In [ ]:
hml.info()

total_bedrooms has some missing values. We have to handle this.

Missing Values are handled using an imputer.

Imputation is a statistical process of replacing null values (missing values) with statistics of the available values. In advanced applications, we even use another ML model (say regression, or RF) as an imputer. See: https://en.wikipedia.org/wiki/Imputation_(statistics)

In this particular case, the number of missing entries are small, and only the total_bedrooms feature is missing. So either we can drop the rows where the total_bedrooms is missing; or we can drop the feature total_bedrooms.

If we don't want to lose valuable datapoints, typically, we use replacement with mean or median as an imputer. Here, we will simply use median and sklearn.impute.SimpleImputer

SimpleImputer with median only works with numerical data.

So we will only use numerical data to perform the imuptation

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

hml_num = hml.drop("ocean_proximity", axis=1)

imputer.fit(hml_num)

X = imputer.transform(hml_num) # Imputer returns a numpy array. So we need to transform it back to a pandas df

hml_num_tr = pd.DataFrame(data=X,columns=hml_num.columns,index=hml_num.index)

In [ ]:
hml_num_tr.info()

So far, so good. We have successfully removed the nan entries. You can verify if everything went correct by looking at the rows with nan entries in hml and corresponding rows in hml_num_tr.

Before we deal with categorical data, let us add the features we engineered.

In [ ]:
hml_num_tr["bedrooms_per_room"] = hml_num_tr["total_bedrooms"]/hml_num_tr["total_rooms"]
hml_num_tr["bedrooms_per_house"] = hml_num_tr["total_bedrooms"]/hml_num_tr["households"]
hml_num_tr["rooms_per_house"] = hml_num_tr["total_rooms"]/hml_num_tr["households"]

**Handle Categorical Data**

Ocean Proximity is a categorical variable. It must be handled.

There are several ways to handle catergorical data. We will use a popular technique called as OneHotEncoder.

We can use sklearn.preprocessing.OneHotEncoder

Alternatively we can use pandas.get_dummies. See: https://towardsdatascience.com/what-is-one-hot-encoding-and-how-to-use-pandas-get-dummies-function-922eb9bd4970

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()

hml_cat = hml[["ocean_proximity"]] # Use two square brackets as fit_transform expects a df. With singe square bracket, a series is returned

hml_cat_1hot = cat_encoder.fit_transform(hml_cat)

In [ ]:
hml_cat_1hot

#this is a sparse array. We must use .toarray() to convert into a full matrix that can be concatenated to the numerical features.

In [ ]:
X_train = np.concatenate([hml_num_tr.to_numpy(),hml_cat_1hot.toarray()],axis=1)

y_train = hml_labels.values

In [ ]:
X_train

**Scaling**

We observed that the features have different scales. So, we perform a standard scaling operation to bring all data to the same scale.

Usually, targets don't need to be scaled. There is no harm in scaling the target separately, but we don't do it.

Task: Use sklearn.preprocessing.StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scl = scaler.fit_transform(X_train)

With this, we have the X_train_scl and y_train that can be used for machine learning.

This X_train does not have the new features that we engineered.

But, we have done a lot of work to transform our dataframe to features. We must do the same for future data, including test data.

For this purpuse, we create a "pipeline" and use "custom_transformation"

# Custom Transformation and Pipeline

1. Handle missing numerical data
2. Add new features columns
3. Handled the categorical data
4. Scale

**First**: Create a custom transformation to add new attribute through feature combinations

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

col_names = ["total_rooms","total_bedrooms","population","households"]

rooms_indx, bedrooms_indx, pop_indx, house_indx = [housing.columns.get_loc(ii) for ii in col_names] #We are using Python list comprehension

class EngineeredFeatureAdder(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    rooms_per_household = X[:,rooms_indx]/X[:,house_indx]
    bedrooms_per_household = X[:,bedrooms_indx]/X[:,house_indx]
    population_per_household = X[:,pop_indx]/X[:,house_indx]
    bedrooms_per_room = X[:,bedrooms_indx]/X[:,rooms_indx]
    return np.c_[X,rooms_per_household,bedrooms_per_household,population_per_household,bedrooms_per_room]

# Note: Here, we are adding all engineered features. This is probably not a good idea. We are simply showing an example.

The input to the cusom transformer EngineeredFeatureAdder is a numpy array. We cannot pass in a pandas dataframe here.

If we need a pandas dataframe later, then we have to create a pipeline with a pandasizer, a function that takes the output of the cusomt transformer and returns a pandas df.

Exercise: Create a pipeline with a pandasizer

# Transformation Pipeline

Build a pipeline for the numerical attributes, and categorical attributes separately.

Then we will combine them together using column transformer

We must impute and scale.



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
                         ('imputer',SimpleImputer(strategy="median")),
                         ('feat_eng',EngineeredFeatureAdder()),
                         ('scaler',StandardScaler())
                         ])

cat_pipeline = Pipeline([
                         ('encoder',OneHotEncoder())
                         ])

from sklearn.compose import ColumnTransformer

num_cols = list(hml_num) # Alternatively: hml_num.columns.values.tolist()
cat_cols = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
                                   ('num',num_pipeline,num_cols),
                                   ('cat',cat_pipeline,cat_cols)
                                   ])

hml_prepared = full_pipeline.fit_transform(hml)

# ML Modeling

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

lin_reg.fit(hml_prepared,hml_labels)

In [ ]:
from sklearn.metrics import mean_squared_error
hml_test = df_test_strat.drop("median_house_value", axis=1) # drop labels for training set
hml_test_labels = df_test_strat["median_house_value"].copy()
X_test = full_pipeline.transform(hml_test)
y_test_predict = lin_reg.predict(X_test)
mean_squared_error(hml_test_labels,y_test_predict,squared=False)


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lin_reg,hml_prepared,hml_labels,scoring="neg_mean_squared_error",cv=5)


In [ ]:
np.sqrt(-scores)

# Fine Tune your model

The cross_val_score shows that the training is stable. The validation error is not varying widely.

For linear regression, there is no further hyper-parameter tuning.

But if we go for ElasticNet, or Random Forest, or XGBoost there is hyper-parameter tuning.

Hyper-parameter tuning can be done with GridSearchCV or RandomizedSearchCV



In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

enet = ElasticNet()

params = {'alpha': [0.1, 1, 10, 100], 'l1_ratio': [0.25,0.5,0.75]}

regressor = GridSearchCV(enet,params,cv=5)

regressor.fit(hml_prepared,hml_labels)

In [ ]:
regressor.best_params_

In [ ]:
regressor.cv_results_

The best hyper-parameter set can be found from the CV results and used to train a best model.

Setting the parameter refit=True will automatically do the refit for you.

Or you can decide to pick the best three models and create an ensemble.

We will study more about ensembles and voting in future lectures.

In [ ]:
best_model = ElasticNet(alpha=0.1,l1_ratio=0.75)

best_model.fit(hml_prepared,hml_labels)

In [ ]:
hml_test = df_test_strat.drop("median_house_value", axis=1) # drop labels for training set
hml_test_labels = df_test_strat["median_house_value"].copy()
X_test = full_pipeline.transform(hml_test)
y_test_predict = best_model.predict(X_test)
mean_squared_error(hml_test_labels,y_test_predict,squared=False)


# Conclusion

The error is slightly on the higher side. An average error of 67k USD might not be acceptable for house values.

Further modeling, feature engineering, and discussions with stakeholders is needed before finalizing a model for this study.

In [ ]:
from IPython.display import clear_output



clear_output()